In [458]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import missingno as msno
import plotly.express as px # plotting geo data
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Train

## Data

In [623]:
data = pd.read_csv("train.csv", header=0)
data = data.drop(columns=['is_promoted','sticker','price_drop_date'])

train, test = train_test_split(data.sort_values('added_time'),test_size=0.2, shuffle=False)

train = train.reset_index()
train['energy_label'] = train['energy_label'].astype('category')
train['new_building'] = train['new_building'].astype('bool')
train['postcode'] = train['postcode'].astype('category')
# train.dtypes

In [624]:
missing_values = round(train.isna().sum()/len(train)*100,2)
missing_checker = train.isna()*1
# missing_checker = missing_checker.drop(columns=missing_values.index[(missing_values>30) | (missing_values==0)].tolist()).drop(columns='energy_label').reset_index()
missing_checker = missing_checker.drop(columns=missing_values.index[missing_values==0].tolist()).reset_index(drop=True)

In [625]:
train['bedrooms_cat'] = pd.cut(train['bedrooms'], bins=[-1,0,1,2,3,4,5,6,float('inf')], labels=['0','1','2','3','4','5','6','7+'])

In [626]:
house_map = {
    'Apartments & Flats': [
        'Appartement', 'Gelijkvloers app.', 'Duplex', 'Triplex', 'Dakappartement',
        'Penthouse', 'Serviceflat', 'Assistentie-appartement', 'Studio', 'Studio met slaaphoek', 
        'App. vrij beroep', 'Appartementsgebouw'
    ],
    'Single-Family Houses': [
        'Eengezinswoning', 'Woning', 'Villa', 'Villa-landhuis', 'Moderne villa',
        'Cottage', 'Bungalow', 'Koppelwoning', 'Koppelvilla', 'Hoekwoning', 'Rijwoning', 'Bel-étage', 'Burgerswoning'
    ],
    'Historical & Luxurious Homes': [
        'Herenhuis', 'Herenwoning', 'Uitzonderlijke woning', 'Kasteel', 'Pastorijwoning'
    ],
    'Farm & Rural Houses': [
        'Hoeve', 'Boerderij', 'Fermette', 'Chalet'
    ],
    'Mixed-Use & Unique Properties': [
        'Gemengd gebruik', 'Arbeiderswoning', 'Kangoeroewoning', 'Woonboot', 'Loft',
        'Split-level', 'Patio woning', 'Buitenverblijf', 'Vakantiewoning'
    ]
}

# Function to categorize houses
def categorize_house(house_type):
    for category, types in house_map.items():
        if house_type in types:
            return category
    return 'Other'


In [627]:
train['house_type'] = train['subtype'].apply(categorize_house)

In [628]:
most_frequent_subtype = train.groupby('bedrooms_cat', observed=False)['subtype'].apply(lambda x: x.value_counts().idxmax())
train['subtype']=train.groupby('bedrooms_cat', observed=False)['subtype'].transform(lambda x: x.fillna(most_frequent_subtype[x.name]))

In [629]:
lat_lon_lookup=train.groupby('postcode', observed=False)[['lat','lon']].mean().dropna().reset_index()
lat_lon_lookup.columns = ['postcode','lat_1','lon_1']
lat_lon_lookup['postcode'] = lat_lon_lookup['postcode'].astype('category')

postcode_list = pd.read_csv("BE.txt", sep="\t", header=None)[[1,9,10]]
postcode_list.columns = ['postcode','lat_2','lon_2']
postcode_list['postcode']=postcode_list['postcode'].astype('category')
postcode_list = postcode_list.groupby('postcode', observed=False)[['lat_2','lon_2']].mean().reset_index()

In [630]:
train = train.merge(lat_lon_lookup, on=['postcode'], how='left')
train = train.merge(postcode_list, on=['postcode'], how='left')
train['lat'] = train['lat'].fillna(train['lat_1']).fillna(train['lat_2'])
train['lon'] = train['lon'].fillna(train['lon_1']).fillna(train['lon_2'])
train = train.drop(columns=['lat_1','lat_2','lon_1','lon_2'])

In [631]:
clf_geo = EllipticEnvelope(contamination=0.0002, support_fraction=0.9)
clf_geo.fit(train[['lat','lon']])
outliers_geo = clf_geo.predict(train[['lat','lon']])

train['lat'] = train['lat'].where(outliers_geo==1, np.nan)
train['lon'] = train['lon'].where(outliers_geo==1, np.nan)

lat_lon_lookup=train.groupby('postcode', observed=False)[['lat','lon']].mean().dropna().reset_index()
lat_lon_lookup.columns = ['postcode','lat_1','lon_1']

train = train.merge(lat_lon_lookup, on=['postcode'], how='left')
train = train.merge(postcode_list, on=['postcode'], how='left')
train['lat'] = train['lat'].fillna(train['lat_1']).fillna(train['lat_2'])
train['lon'] = train['lon'].fillna(train['lon_1']).fillna(train['lon_2'])
train = train.drop(columns=['lat_1','lat_2','lon_1','lon_2'])

In [632]:
# fig = px.scatter_geo(train, lat='lat', lon='lon', scope='europe')
# fig.update_geos(showcountries=True, showcoastlines=True)
# fig.show()

In [633]:
clf_area = EllipticEnvelope(contamination=0.0002, support_fraction=0.9)
outliers_area = clf_area.fit_predict(train[['area']].dropna())
train['area'] = train['area'].where(~train['id'].isin(train[['area','id']].dropna()[outliers_area==-1]['id']), np.nan)
print(train['area'].max())

1350.0


In [634]:
sub_bed_area_lookup = train.groupby(['bedrooms_cat','subtype'], observed=False)['area'].median().dropna().reset_index()
sub_bed_area_lookup.columns = ['bedrooms_cat','subtype', 'area_1']
bed_ht_area_lookup = train.groupby(['bedrooms_cat', 'house_type'], observed=False)['area'].median().dropna().reset_index()
bed_ht_area_lookup.columns = ['bedrooms_cat','house_type', 'area_2']
bed_area_lookup = train.groupby(['bedrooms_cat'], observed=False)['area'].median().dropna().reset_index()
bed_area_lookup.columns = ['bedrooms_cat', 'area_3']

train = train.merge(sub_bed_area_lookup, on=['bedrooms_cat','subtype'], how='left')
train = train.merge(bed_ht_area_lookup, on=['bedrooms_cat','house_type'], how='left')
train = train.merge(bed_area_lookup, on=['bedrooms_cat'], how='left')
train['area'] = train['area'].fillna(train['area_1']).fillna(train['area_2']).fillna(train['area_3'])
train = train.drop(columns=['area_1','area_2','area_3'])

In [635]:
el_lookup = train.groupby(['energy_label','province'], observed=False)['energy_value'].median().dropna().reset_index()
el_lookup.columns = ['energy_label', 'province', 'energy_value_1']

train = train.merge(el_lookup, on=['energy_label', 'province'], how='left')
train['energy_value'] = train['energy_value'].fillna(train['energy_value_1'])
train = train.drop(columns='energy_value_1')

In [636]:
ev_imputer = KNNImputer(n_neighbors=10, weights='distance')
imputed_data = ev_imputer.fit_transform(train[['energy_value','area','is_appartment','new_building', 'lat','lon']])
# df_imputed = pd.DataFrame(imputed_data)
train['energy_value']= pd.DataFrame(imputed_data)[0]

In [637]:
# plt.scatter(x=train['energy_value'],y=train['area'],c=missing_checker['energy_value'])
# plt.xscale('log')
# plt.yscale('log')
# plt.xlim()

In [638]:
train['advertiser'] = train['advertiser'].fillna('Other')
train['advertiser'] = train['advertiser'].astype('category')

In [639]:
missing_checker.columns = ['area_miss', 'lat_miss', 'lon_miss', 'advertiser_miss', 'subtype_miss', 'energy_value_miss', 'energy_label_miss']
missing_checker = (missing_checker==1)
train = pd.concat([train, missing_checker], axis=1)
train['house_type'] = train['house_type'].astype('category')
train['province']=train['province'].astype('category')

## Model

### KNN

In [640]:
# X_train, y_train = train[['is_appartment','area','lat','lon','foto_amount','energy_value','house_type']].copy(),train['price'].copy()

# X_train_encoded = pd.get_dummies(X_train, columns=['house_type'])

# scaler = StandardScaler()
# X_train_encoded[['area','lat','lon','energy_value','foto_amount']] = scaler.fit_transform(X_train_encoded[['area','lat','lon','energy_value','foto_amount']])

In [641]:
# scores_KNN = []
# for i in range(2,60,2):
#     neigh_model = KNeighborsRegressor(n_neighbors=i, weights='distance')
#     scores_KNN.append(cross_val_score(neigh_model, X_train_encoded, y_train, cv=5, scoring='neg_mean_absolute_percentage_error')*-1)

# scores_KNN = pd.DataFrame(scores_KNN)
# plt.scatter((scores_KNN.index+1)*2, scores_KNN.apply(np.mean, axis=1))

In [642]:
# neigh_model_final = KNeighborsRegressor(n_neighbors=14, weights='distance')
# neigh_model_final.fit(X_train_encoded,y_train)

### XGBoost

In [649]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=['added_time','postcode','energy_label','subtype','bedrooms','id','index','price']),train['price'], shuffle=False, test_size=0.2)

In [650]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dval = xgb.DMatrix(X_val,label=y_val, enable_categorical=True)

In [651]:
evallist = [(dval, 'val')]

In [659]:
param_xgb = {'max_depth': 6, 'eta': 0.1, 'objective': 'reg:squarederror', 'reg_lambda':10, 'reg_alpha':10,}

xgb_regressor = xgb.train(
    params=param_xgb,
    dtrain=dtrain,
    num_boost_round=500
)
xgb_regressor.eval_set([(dval,'val')])

'[0]\tval-rmse:114558.26176200545160100'

In [647]:
X_train, y_train = train.drop(columns=['added_time','postcode','energy_label','subtype','bedrooms','id','index','price']).copy(),train['price'].copy()
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

param_xgb = {'max_depth': 6, 'eta': 0.1, 'objective': 'reg:squarederror', 'reg_lambda':10, 'reg_alpha':10,}

xgb_regressor_final = xgb.train(
    params=param_xgb,
    dtrain=dtrain,
    num_boost_round=500
)

# Test

## Data

In [604]:
# test = pd.read_csv("test.csv", header=0)
test = test.reset_index()
# test = test.drop(columns=['is_promoted','sticker','price_drop_date'])
test['energy_label'] = test['energy_label'].astype('category')
test['new_building'] = test['new_building'].astype('bool')
test['postcode'] = test['postcode'].astype('category')

test['bedrooms_cat'] = pd.cut(test['bedrooms'], bins=[-1,0,1,2,3,4,5,6,float('inf')], labels=['0','1','2','3','4','5','6','7+'])
test['house_type'] = test['subtype'].apply(categorize_house)

In [ ]:
missing_values = round(test.isna().sum()/len(test)*100,2)
missing_checker = test.isna()*1
# missing_checker = missing_checker.drop(columns=missing_values.index[(missing_values>30) | (missing_values==0)].tolist()).drop(columns='energy_label').reset_index()
missing_checker = missing_checker.drop(columns=missing_values.index[missing_values==0].tolist()).reset_index(drop=True)

missing_checker.columns = ['area_miss', 'lat_miss', 'lon_miss', 'advertiser_miss', 'subtype_miss', 'energy_value_miss', 'energy_label_miss']
missing_checker = (missing_checker==1)
test = pd.concat([test, missing_checker], axis=1)

In [606]:
test['advertiser'] = test['advertiser'].fillna('Other')
test['advertiser'] = test['advertiser'].astype('category')

In [607]:
test['subtype']=test.groupby('bedrooms_cat', observed='False')['subtype'].transform(lambda x: x.fillna(most_frequent_subtype[x.name]))

In [608]:
test = test.merge(lat_lon_lookup, on=['postcode'], how='left')
test = test.merge(postcode_list, on=['postcode'], how='left')
test['lat'] = test['lat'].fillna(test['lat_1']).fillna(test['lat_2'])
test['lon'] = test['lon'].fillna(test['lon_1']).fillna(test['lon_2'])

outliers_geo_test = clf_geo.predict(test[['lat','lon']])

test['lat'] = test['lat'].where(outliers_geo_test==1, np.nan)
test['lon'] = test['lon'].where(outliers_geo_test==1, np.nan)

test['lat'] = test['lat'].fillna(test['lat_1']).fillna(test['lat_2'])
test['lon'] = test['lon'].fillna(test['lon_1']).fillna(test['lon_2'])

test = test.drop(columns=['lat_1','lat_2','lon_1','lon_2'])

In [609]:
outliers_area = clf_area.predict(test[['area']].dropna())
test['area'] = test['area'].where(~test['id'].isin(test[['area','id']].dropna()[outliers_area==-1]['id']), np.nan)

test = test.merge(sub_bed_area_lookup, on=['bedrooms_cat','subtype'], how='left')
test = test.merge(bed_ht_area_lookup, on=['bedrooms_cat','house_type'], how='left')
test = test.merge(bed_area_lookup, on=['bedrooms_cat'], how='left')
test['area'] = test['area'].fillna(test['area_1']).fillna(test['area_2']).fillna(test['area_3'])
test = test.drop(columns=['area_1','area_2','area_3'])

In [610]:
test = test.merge(el_lookup, on=['energy_label','province'], how='left')
test['energy_value'] = test['energy_value'].fillna(test['energy_value_1'])
test = test.drop(columns='energy_value_1')

imputed_data = ev_imputer.fit_transform(test[['energy_value','area','is_appartment','new_building', 'lat','lon']])
test['energy_value']= pd.DataFrame(imputed_data)[0]

In [612]:
test['house_type'] = test['house_type'].astype('category')
test['province']=test['province'].astype('category')

## KNN

In [ ]:
# X_test = test[['is_appartment','area','lat','lon','foto_amount','energy_value','house_type']].copy()

# X_test_encoded = pd.get_dummies(X_test, columns=['house_type'])

# scaler = StandardScaler()
# X_test_encoded[['area','lat','lon','energy_value','foto_amount']] = scaler.fit_transform(X_test_encoded[['area','lat','lon','energy_value','foto_amount']])
# X_test_encoded.isna().apply(sum)
# y_test = test['price'].copy()
# y_pred_test = neigh_model_final.predict(X_test_encoded)

## XGBoost

In [ ]:
X_test, y_test = test.drop(columns=['added_time','postcode','energy_label','subtype','bedrooms','id','index','price']).copy(),test['price'].copy()
dtest = xgb.DMatrix(X_test, enable_categorical=True)

y_pred_test=xgb_regressor_final.predict(dtest)

## Error bars

In [615]:
mean_error_1=(y_pred_test/y_test-1).mean()
error_bound_1 = (y_pred_test/y_test-1).std()/len(y_pred_test)**(1/2)*1.96

mean_error_2=abs(y_pred_test/y_test-1).mean()
error_bound_2 = abs(y_pred_test/y_test-1).std()/(len(y_pred_test)**(1/2))*1.96

mean_error_3=((y_pred_test-y_test)**2).mean()**(1/2)
error_bound_3 = abs(y_pred_test-y_test).std()/(len(y_pred_test)**(1/2))*1.96

# Submission metrics

In [510]:
y_pred_final = pd.DataFrame()
y_pred_final['id']=test['id']
y_pred_final['lower']=y_pred_test*(1-mean_error_1-error_bound_1)
y_pred_final['upper']=y_pred_test*(1-mean_error_1+error_bound_1)
y_pred_final['pred']=y_pred_test*(1-mean_error_1)

y_pred_final.to_csv(f"xgb_simple_1.csv", index=False)

In [513]:
y_pred_final = pd.DataFrame()
y_pred_final['id']=test['id']
y_pred_final['lower']=y_pred_test*(1-mean_error_2-error_bound_2)
y_pred_final['upper']=y_pred_test*(1+mean_error_2+error_bound_2)
y_pred_final['pred']=y_pred_test

y_pred_final.to_csv(f"xgb_simple_2.csv", index=False)

In [512]:
y_pred_final = pd.DataFrame()
y_pred_final['id']=test['id']
y_pred_final['lower']=np.maximum(y_pred_test-mean_error_3-error_bound_3,0)
y_pred_final['upper']=y_pred_test+mean_error_3+error_bound_3
y_pred_final['pred']=y_pred_test

y_pred_final.to_csv(f"xgb_simple_3.csv", index=False)